In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import cv2

In [ ]:
Data_folder = '../input/covid-face-mask-detection-dataset/New Masks Dataset'
train_f = os.path.join(Data_folder,'Train')
test_f = os.path.join(Data_folder,'Test')
valid_f = os.path.join(Data_folder,'Validation')
mask_train = os.path.join(train_f,'Mask')
nomask_train = os.path.join(train_f,'Non Mask')

In [ ]:
mask_train_name = os.listdir(mask_train)
print(mask_train_name[:25])
nomask_train_name = os.listdir(nomask_train)
print(nomask_train_name[:25])

In [ ]:
rows = 6
cols = 6
plt.figure(figsize=(20,20))

with_mask = []
for i in mask_train_name[1:10]:
  with_mask.append(os.path.join(mask_train,i))

without_mask = []
for i in nomask_train_name[1:10]:
  without_mask.append(os.path.join(nomask_train,i))

print(with_mask)
print(without_mask)

merged = with_mask+without_mask

for i in range(0,len(merged)):
  data = merged[i].split('/',4)[4]
  sp = plt.subplot(6,6,i+1)
  sp.axis('Off')
  image = mpimg.imread(merged[i])
  sp.set_title(data,fontsize=10)
  plt.imshow(image,cmap='gray')

plt.show()
  


# Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range = 0.3,
                                   rotation_range = 35,
                                   horizontal_flip = True
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_f,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )
test_generator = test_datagen.flow_from_directory(test_f,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )
valid_generator = validation_datagen.flow_from_directory(valid_f,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )

In [ ]:
train_generator.class_indices

In [ ]:
train_generator.image_shape

# Building CNN Model

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='SAME',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))



model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()



#model.add(Conv2D(128,(3,3),padding='SAME',activation='relu'))
#model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.5))

#model.add(Conv2D(256,(3,3),padding='SAME',activation='relu'))
#model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.5))

In [ ]:
model.compile(Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,epochs = 50,validation_data = valid_generator)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['trainig','validation'])
plt.title('loss')
plt.xlabel('epoch')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['trainig','validation'])
plt.title('accuracy')
plt.xlabel('epoch')

In [ ]:
test_loss , test_acc = model.evaluate(test_generator)
print('test acc :{} test loss:{}'.format(test_acc,test_loss))

In [ ]:
model.save('1.model.model')

In [ ]:
model = load_model('1.model')